In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels as sm
#%matplotlib inline
%matplotlib notebook

In [3]:
# Data Aggregation and Group Operations
# group means
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'], 
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randn(5), 
                  'data2': np.random.randn(5)})
print(df)
grouped = df['data1'].groupby(df['key1']) 
#df.groupby('key1')['data1']
grouped.mean()

  key1 key2     data1     data2
0    a  one  0.400902  0.030139
1    a  two -0.015870 -1.398730
2    b  one  1.415469 -0.081304
3    b  two -0.476871 -1.274454
4    a  one  0.427126 -0.541076


key1
a    0.270719
b    0.469299
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
print(means)
means.unstack()

key1  key2
a     one     0.414014
      two    -0.015870
b     one     1.415469
      two    -0.476871
Name: data1, dtype: float64


key2,one,two
key1,,
a,0.414014,-0.015870
b,1.415469,-0.476871


In [6]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [7]:
# Iterating over groups
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.400902  0.030139
4    a  one  0.427126 -0.541076
('a', 'two')
  key1 key2    data1    data2
1    a  two -0.01587 -1.39873
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.415469 -0.081304
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.476871 -1.274454


In [8]:
# grouping with dicts
people = pd.DataFrame(np.random.randn(5, 5), 
                      columns=['a', 'b', 'c', 'd', 'e'], index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan
print(people)
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e':'red', 'f':'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()


               a         b         c         d         e
Joe     2.895502 -1.678433  0.670978  0.204575  0.369203
Steve   1.745044 -0.803243  0.250472  0.203721 -0.337534
Wes    -0.128755       NaN       NaN -0.234885 -1.356052
Jim     0.110851  0.167852 -0.280249 -0.222604 -0.137458
Travis  2.612835 -0.033994  1.451044  1.182484  0.545978


,blue,red
Joe,0.875553,1.586272
Steve,0.454193,0.604267
Wes,-0.234885,-1.484807
Jim,-0.502853,0.141246
Travis,2.633528,3.124819


In [9]:
map_series = pd.Series(mapping)
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [17]:
# Data aggregation
# create your own agg function
grouped = df.groupby('key1')
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.442995,1.42887
b,1.892340,1.19315


In [12]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.270719  0.248539 -0.015870  0.192516  0.400902  0.414014   
b      2.0  0.469299  1.338086 -0.476871 -0.003786  0.469299  0.942384   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.427126   3.0 -0.636556  0.719204 -1.398730 -0.969903 -0.541076   
b     1.415469   2.0 -0.677879  0.843684 -1.274454 -0.976166 -0.677879   

                          
           75%       max  
key1                      
a    -0.255468  0.030139  
b    -0.379591 -0.081304

In [22]:
# multiple function
tips = pd.read_csv('examples\Tips.txt')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
grouped = tips.groupby(['day', 'smoker'], as_index=False) # ignore the original index
grouped_pct = grouped['tip_pct']
grouped_pct.agg(['mean', 'std', peak_to_peak]) # passed a list of agg functions and evaluate indepedently

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [ ]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)]) # foo and bar as column names

In [ ]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions) # pass multiple agg functions in a list

In [23]:
grouped.agg({'tip' : np.max, 'size' : 'sum'}) # apply different functions to columns

,day,smoker,tip,size
0,Fri,No,3.50,9
1,Fri,Yes,4.73,31
2,Sat,No,9.00,115
3,Sat,Yes,10.00,104
4,Sun,No,6.00,167
5,Sun,Yes,6.50,49
6,Thur,No,6.70,112
7,Thur,Yes,5.00,40


In [25]:
# Split-apply-combine
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
tips.groupby('smoker', group_keys=False).apply(top)
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill') # pass specific arguments

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [27]:
# Quantile and bucket
frame = pd.DataFrame({'data1': np.random.randn(1000), 
                     'data2': np.random.randn(1000)})
quantiles = pd.cut(frame.data1, 4) # 4 equal-length buckets, use qcut to get equal-size buckets
def get_stats(group):
    return {'min': group.min(), 'max':group.max(), 
           'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quantiles) # can group data directly by quantile objects
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.056, -1.572]",57.0,2.266561,0.148767,-3.432478
"(-1.572, -0.0935]",409.0,2.675682,0.036134,-2.505556
"(-0.0935, 1.385]",458.0,3.349788,0.000156,-2.972482
"(1.385, 2.863]",76.0,1.887468,-0.096913,-1.980498


In [28]:
# random sampling and permutation
# playing cards
suits = ['H', 'S', 'C', 'D'] # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)

# draw 5 random cards
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

AC      1
4S      4
JS     10
10S    10
9D      9
dtype: int64

In [31]:
# draw 2 random cards from each suit
get_suit = lambda card: card[-1]
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

7C     7
AC     1
AD     1
9D     9
8H     8
2H     2
KS    10
AS     1
dtype: int64

In [34]:
# Pivot tables
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], 
                 columns='smoker', margins=True) # calculate group means by default

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [35]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], 
                 columns='smoker', aggfunc=len, margins=True) # len/count calculate group sizes

size            tip_pct             
smoker          No   Yes  All      No   Yes    All
time   day                                        
Dinner Fri     3.0   9.0   12     3.0   9.0   12.0
       Sat    45.0  42.0   87    45.0  42.0   87.0
       Sun    57.0  19.0   76    57.0  19.0   76.0
       Thur    1.0   NaN    1     1.0   NaN    1.0
Lunch  Fri     1.0   6.0    7     1.0   6.0    7.0
       Thur   44.0  17.0   61    44.0  17.0   61.0
All          151.0  93.0  244   151.0  93.0  244.0

In [37]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], 
                 columns='smoker', aggfunc='mean', fill_value=0) # fill NA values with 0

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000  0.000000  0.159744  0.000000
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [38]:
# cross tabulations
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244